In [ ]:
# Let's Look at the true cosmics

In [ ]:
import ROOT
import numpy as np
import os
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from mpl_toolkits.mplot3d.art3d import Poly3DCollection
import sys
import multiprocessing as mp
import uproot
import pandas as pd
import pickle
import h5py
import gzip
import math
import timeit


#MC CV Sample --> High Stats
#infile = "../NTuples/mc_production_with_fmatch_ntuple_more_stats.root"
infile = "../NTuples/mc_production_true_particles_ntuple_v1.root"

In [ ]:
inFile = uproot.open(infile)

inFileROOT = ROOT.TFile.Open(infile, "READ")
#h_tot_pot = inFileROOT.Get("TotalPOT")
h_tot_pot = inFileROOT.Get("TOTPOT_Clone")
TOT_POT = h_tot_pot.GetBinContent(1)
inFileROOT.Close()
TOT_POT = f"{TOT_POT:.2e}"
print("Total POT", TOT_POT)

#slc_tree = inFile["slc_truth_tree"]
part_tree1 = inFile["part_tree1"]
part_tree2 = inFile["part_tree2"]
part_tree3 = inFile["part_tree3"]


part_df1 = part_tree1.arrays(part_tree1.keys(), library="pd")
part_df2 = part_tree2.arrays(part_tree2.keys(), library="pd")
part_df3 = part_tree3.arrays(part_tree3.keys(), library="pd")


part_df1[:2]

In [ ]:
part_df2["pdg"] = part_df1["pdg"]
part_df3["pdg"] = part_df1["pdg"]
part_df2["interaction_id"] = part_df1["interaction_id"]
part_df3["interaction_id"] = part_df1["interaction_id"]

part_df1 = part_df1.query("(pdg == 13 or pdg == -13) and interaction_id == -1")
part_df2 = part_df2.query("(pdg == 13 or pdg == -13) and interaction_id == -1")
part_df3 = part_df3.query("(pdg == 13 or pdg == -13) and interaction_id == -1")

part_df1[:4]

In [ ]:
B = np.arange(-0.5, 1.5, 1)

plt.hist(part_df1["crosses_tpc"].values, bins=B)
plt.show()

In [ ]:
evt_tree = inFile["event_tree"]
evt_df = evt_tree.arrays(evt_tree.keys(), library="pd")
evt_df[:2]

In [ ]:
runs = evt_df["run"].values
subruns = evt_df["subrun"].values


data = {'run': runs,
        'subrun': subruns,
       }

header_df = pd.DataFrame(data)
header_df = header_df.drop_duplicates()

header_df[:2]

In [ ]:
def get_genevt(row):
    r = row["run"]
    sr = row["subrun"]
    g = evt_df.query("run == "+str(r) + " and subrun == "+str(sr))["ngenevt"].values
    if g[-1] != g[0]:
        print("diff ?")
    return g[0]


header_df["genevt"] = header_df.apply(get_genevt, axis=1)
header_df[:3]

In [ ]:
plt.hist(header_df["genevt"].values, bins=100)
plt.show()

In [ ]:
part_df3[:4]

In [ ]:
tpc_cosmics = part_df3.query("start_x > -8000 and start_y > -8000 and start_z > -8000")

print("N cosmics ", tpc_cosmics.shape[0])

In [ ]:
print(np.sum(header_df["genevt"].values))

In [ ]:
print(tpc_cosmics.shape[0]/(3*np.sum(header_df["genevt"].values)))

In [ ]:
evt_df = event_df.drop_duplicates()
evt_df[:2]

In [ ]:

# Loop over events
for num in range(event_df.shape[0]):
    if num % 10000 == 0:
        print("Analyzing event", num)
        
    row = event_df.iloc[num]
    r, sr, e = row["run"], row["subrun"], row["evt"]
    
    m1 = (slc_av_reco_df["run"].values == r)
    m2 = (slc_av_reco_df["subrun"].values == sr)
    m3 = (slc_av_reco_df["evt"].values == e)
    m4 = (slc_av_reco_df["fmatch_time"].values <= 2.1)
    m5 = (slc_av_reco_df["fmatch_time"].values >= 0)
    m = m1 & m2 & m3 & m4 & m5
 
    AV_slc_ids = np.array(slc_av_reco_df["slc"].values)[m]
    if len(AV_slc_ids) > 0:
        N_AV += 1
        
        #print("AV Slice IDs")
        #print(AV_slc_ids)

        N_CRT = 0
        for s in AV_slc_ids:
            m1 = (trk_av_df1["run"].values == r)
            m2 = (trk_av_df1["subrun"].values == sr)
            m3 = (trk_av_df1["evt"].values == e) 
            m4 = (trk_av_df1["slc"].values == s)
            m = m1 & m2 & m3 & m4
            crt_hits = np.array(trk_av_df1["N_CRT_AV"].values)[m]
            n_cont = 0